# Domain Modeling by Kotlin

In [1]:
%use test

In [2]:
import kotlin.test.Test
import kotlin.test.assertEquals

In [79]:
fun test(instance: Any) {
    val klass: kotlin.reflect.KClass<out Any> = instance::class
    val RED_BOLD = "\u001B[1;31m"
    val BLUE_BOLD = "\u001B[3;31m"    
    val RESET = "\u001B[0m"
    
    fun filterStack(it: String): Boolean {
        val line = it.trim()
        return (
            line.indexOf("TestRunner.run") < 0 &&
            line.indexOf("_jupyter.<init>") < 0 &&
            line.startsWith("at Line") &&
            true
        )
    }
    
    klass.members.filter { 
        it.annotations.size > 0 && 
        it.annotations.elementAt(0).annotationClass == org.junit.Test::class
    }.forEach {
        try {
            it.call(instance)
        } catch(e: java.lang.reflect.InvocationTargetException) {
            println("${BLUE_BOLD}${it.name}${RESET}")
            val cause = e.cause.toString()
            val idx = cause.indexOf(":")
            val (name, reason) = listOf(cause.substring(0, idx), cause.substring(idx+1))
            println("${name}:${RED_BOLD}${reason}${RESET}")
            val trace = e.stackTraceToString().split('\n').filter { filterStack(it) }
            println(trace.joinToString("\n"))
            println()
        }
    }
}

In [80]:
fun helloWorld(name: String = "World"): String {
    return "Hello, ${name}!" 
}

class HelloTest {
    @Test fun helloWorldReturnsPersonalizedMessage() {
        assertEquals("Hello, Molly2!", helloWorld("Molly"))
    }
    
    @Test fun alwaysFail() {
        assertNotEquals(false, false)
    }
}

test(HelloTest())

alwaysFail
java.lang.AssertionError: Values should be different. Actual: false
	at Line_86_jupyter.test(Line_86.jupyter.kts:22)
	at Line_87_jupyter$HelloTest.alwaysFail(Line_87.jupyter.kts:11)

helloWorldReturnsPersonalizedMessage
org.junit.ComparisonFailure: expected:<Hello, Molly[2]!> but was:<Hello, Molly[]!>
	at Line_86_jupyter.test(Line_86.jupyter.kts:22)
	at Line_87_jupyter$HelloTest.helloWorldReturnsPersonalizedMessage(Line_87.jupyter.kts:7)

